- 환경변수 설정

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [2]:
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
PINECONE_API_KEY

'pcsk_3RAn2d_6bkVakuPSMkobvFRxMZLjKBcmNWLGBdiLCAH9wvaDR6xFNKjvbCh8MgG7RQSU7H'

- Pinecone 클라이언트 초기화

In [3]:
from pinecone import Pinecone

# Pinecone 클라이언트를 초기화합니다.
# PINECONE_API_KEY는 환경 변수에서 가져온 API 키입니다.
pc = Pinecone(api_key=PINECONE_API_KEY)

In [4]:
# Pinecone 클라이언트를 사용하여 현재 사용 가능한 모든 인덱스의 목록을 반환합니다
pc.list_indexes()

[
    {
        "name": "test-embedding-3d",
        "metric": "cosine",
        "host": "test-embedding-3d-dksb1qh.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 3,
        "deletion_protection": "disabled",
        "tags": null
    },
    {
        "name": "quickstart",
        "metric": "cosine",
        "host": "quickstart-dksb1qh.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 1024,
        "deletion_protection": "disabled",
        "tags": null
    },
    {

In [6]:
# 사용할 인덱스 이름을 지정합니다.
index_name = 'wiki'

# 지정한 이름으로 Pinecone 인덱스를 가져옵니다.
index = pc.Index(index_name)

# 인덱스의 통계 정보를 설명합니다.
index.describe_index_stats()

c:\Users\C118\miniconda3\envs\llmEnv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'wiki-ns1': {'vector_count': 1080}},
 'total_vector_count': 1080,
 'vector_type': 'dense'}

검색하기 (Pinecone)

- 질문을 임베딩(벡터로 만들기)

In [15]:
from langchain_openai import OpenAIEmbeddings
 # OpenAIEmbeddings 객체를 초기화합니다.
 # 모델은 "text-embedding-3-small"을 사용하고, API 키는 OPENAI_API_KEY를 사용합니다.
embedding = OpenAIEmbeddings(model="text-embedding-3-small")
# 질문 리스트를 정의합니다.
# question = ["U.S television channel G4"]
question = ["미국의 텔레비전 채널 G4에 대하여"]
# 질문을 임베딩하여 벡터로 변환합니다.
embedded_question = embedding.embed_documents(question)
# 임베딩된 질문 벡터를 출력합니다.
print(embedded_question)
print(len(embedded_question[0])) # 1536(dimension)

[[0.027905907481908798, 0.014057626947760582, -0.020693914964795113, 0.027926841750741005, 0.009122279472649097, 0.009923030622303486, -0.01191182341426611, 0.011576869525015354, 0.024870380759239197, -0.029580680653452873, 0.027068519964814186, 0.03856164962053299, -0.04517700523138046, -0.0042078676633536816, -0.023467758670449257, -0.0733550637960434, -0.005950678139925003, -0.03167414665222168, -0.0018684187671169639, -0.01943783648312092, -0.03154853731393814, 0.01117911096662283, -0.011555934324860573, -0.001417015097104013, -0.010739482939243317, -0.018589980900287628, 0.025477485731244087, 0.00010900745110120624, -0.025289073586463928, -0.009258355014026165, -0.006217595189809799, -0.03726370260119438, 0.029894700273871422, -0.06711653620004654, 0.04262297600507736, -0.0029962738044559956, 0.003472537500783801, 0.014989219605922699, -0.008871063590049744, 0.03575640544295311, 0.024619165807962418, 0.02698478288948536, 0.01911334879696369, -0.007897602394223213, 0.06456250697374

- 벡터DB 에서 조회(검색)

In [ ]:
# Pinecone 인덱스에서 쿼리를 실행합니다.
query_result = index.query(
    namespace="wiki-ns1",  # wiki-ns1 네임스페이스를 지정합니다.
    vector=embedded_question,  # 임베딩된 질문 벡터를 사용합니다.
    top_k=5,  # 상위 5개의 결과를 반환합니다.
    include_vector=False,  # 벡터를 포함하지 않습니다.
    include_metadata=True  # 메타데이터를 포함합니다.
 )


['61dd886c-9999-40b7-b413-23d300f164cb', 'd9968e4b-16bb-4019-8b42-2130ed415e83', '23a1e974-d00d-445d-80fe-47b4ed30105c', 'cf4d6068-4db0-483a-850c-6c265bf17426', 'a37fb82b-8d83-4e87-9a3f-ad7aad8f9fb6']


In [18]:
print(len(query_result.matches))
print(query_result)

5
{'matches': [{'id': '61dd886c-9999-40b7-b413-23d300f164cb',
              'metadata': {'chunk_id': 0.0,
                           'head_chunk': 'This is a list of television '
                                         'programs formerly broadcast by the '
                                         'U.S. cable television channel G4.\n'
                                         '\n'
                                         'Second iteration\n'
                                         '\n'
                                         'Final programming\n'
                                         '\n'
                                         'Original\n'
                                         '\n'
                                         ' Xplay (2004–2013; 2013–2014 '
                                         '(reruns); 2021–2022)\n'
                                         ' Att',
                           'title': 'List of programs broadcast by G4',
                           'url': 'https

In [19]:
 # 쿼리 결과 query_result 에서 각 매치의 ID를 추출하여 리스트로 만듭니다.
result_ids = [r.id for r in query_result.matches]
print(result_ids)  # 결과 ID를 출력합니다.

['61dd886c-9999-40b7-b413-23d300f164cb', 'd9968e4b-16bb-4019-8b42-2130ed415e83', '23a1e974-d00d-445d-80fe-47b4ed30105c', 'cf4d6068-4db0-483a-850c-6c265bf17426', 'a37fb82b-8d83-4e87-9a3f-ad7aad8f9fb6']


In [20]:
# 쿼리 결과에서 각 매치의 점수와 메타데이터를 출력합니다.
for r in query_result.matches:
  print(r.score, r.metadata)

0.42159462 {'chunk_id': 0.0, 'head_chunk': 'This is a list of television programs formerly broadcast by the U.S. cable television channel G4.\n\nSecond iteration\n\nFinal programming\n\nOriginal\n\n Xplay (2004–2013; 2013–2014 (reruns); 2021–2022)\n Att', 'title': 'List of programs broadcast by G4', 'url': 'https://en.wikipedia.org/wiki/List%20of%20programs%20broadcast%20by%20G4', 'wiki_id': '8125821'}
0.361602813 {'chunk_id': 9.0, 'head_chunk': 'Voyagers! (2013)\n\nProgramming blocks \n Expansion Pack (2002–2003)\n G4 Global Gamer (2003–2004)\n Real Time (2003–2005)\n Anime Unleashed (2004–2006) (originated on TechTV and later moved to G4techTV whe', 'title': 'List of programs broadcast by G4', 'url': 'https://en.wikipedia.org/wiki/List%20of%20programs%20broadcast%20by%20G4', 'wiki_id': '8125821'}
0.356935501 {'chunk_id': 14.0, 'head_chunk': 'Morgan Minute - Hosted by Morgan Webb, this segment shows the latest games designed specifically for women.\n Tech Alert - This segment shows th

검색하기 (LangChain)

In [ ]:
from langchain_pinecone import PineconeVectorStore
# PineconeVectorStore 객체를 초기화합니다.
# 인덱스는 이전에 정의된 index 객체를 사용하고, 임베딩은 embedding 객체를 사용합니다.
# 텍스트 키는 "head_chunk"로 설정합니다.
vector_store = PineconeVectorStore(index=index, embedding=embedding, text_key="head_chunk")
# 질문에 대한 유사도 검색을 수행합니다.
# query는 question 리스트의 첫 번째 요소를 사용하고, 상위 5개의 결과를 반환합니다.
docs = vector_store.similarity_search(query=question[0], k=5, namespace="wiki-ns1")
print(docs)
# 검색된 문서의 메타데이터를 출력합니다.
for doc in docs:
    print(doc.metadata)

[Document(id='61dd886c-9999-40b7-b413-23d300f164cb', metadata={'chunk_id': 0.0, 'title': 'List of programs broadcast by G4', 'url': 'https://en.wikipedia.org/wiki/List%20of%20programs%20broadcast%20by%20G4', 'wiki_id': '8125821'}, page_content='This is a list of television programs formerly broadcast by the U.S. cable television channel G4.\n\nSecond iteration\n\nFinal programming\n\nOriginal\n\n Xplay (2004–2013; 2013–2014 (reruns); 2021–2022)\n Att'), Document(id='d9968e4b-16bb-4019-8b42-2130ed415e83', metadata={'chunk_id': 9.0, 'title': 'List of programs broadcast by G4', 'url': 'https://en.wikipedia.org/wiki/List%20of%20programs%20broadcast%20by%20G4', 'wiki_id': '8125821'}, page_content='Voyagers! (2013)\n\nProgramming blocks \n Expansion Pack (2002–2003)\n G4 Global Gamer (2003–2004)\n Real Time (2003–2005)\n Anime Unleashed (2004–2006) (originated on TechTV and later moved to G4techTV whe'), Document(id='23a1e974-d00d-445d-80fe-47b4ed30105c', metadata={'chunk_id': 14.0, 'title':